In [1]:
import numpy as np

In [18]:
def fit(X_train, Y_train):
    reasult = {}
    #distinct y values as class values(top level keys) 
    class_values = set(Y_train)
    for current_class in class_values:
        #this dictionary will have all possible features
        reasult[current_class] = {}
        #total data
        reasult["total_data"] = len(Y_train)
        #finding the training data which has class as the current class
        current_class_rows = (Y_train == current_class)
        #to get current training data(X_train and Y_train)
        X_train_current =  X_train[current_class_rows]
        Y_train_current =  Y_train[current_class_rows]
        #number of feature in a class
        num_features = X_train.shape[1]
        #total count of training data which belongs to current class
        reasult[current_class]["total_count"] = len(Y_train_current)
        #passing through each feature
        for j in range(1, num_features+1):
            #for each feature create new dictionary
            reasult[current_class][j] = {}
            #counting all possible values of feature j
            all_possible_values = set(X_train[:, j-1])
            for current_value in all_possible_values:
                #count of training data where the class is current class and jth feature value is current _value
                reasult[current_class][j][current_value] = (X_train_current[:, j-1] == current_value).sum()
    return reasult

In [21]:
def predict(dictionary, X_test):#dictionary is nothing but reasult
    y_pred = []
    for x in X_test:
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
    return y_pred

In [15]:
def predictSinglePoint(dictionary, x):
    #finding all possible classes
    classes = dictionary.keys()
    best_p = -1000
    best_class = -1
    first_run =True
    for current_class in classes:
        if (current_class == "total_data"):
            continue
        
        p_current_class = probability(dictionary, x, current_class)
        if(first_run or p_current_class> best_p):
            best_p = p_current_class
            best_class = current_class
        first_run = False
    return best_class

In [16]:
def probability(dictionay, x, current_class):
    output = np.log(dictionay[current_class]["total_count"])-np.log(dictionay["total_data"])
    num_feature = len(dictionay[current_class].keys()) -1
    for j in range(1, num_feature+1):
        xj = x[j -1]
        #for laplace correction add +1 with numerator and with denominator we add how many different values j can take
        count_current_class_with_value_xj = dictionay[current_class][j][xj] +1
        count_current_class = dictionay[current_class]["total_count"] + len((dictionay[current_class][j]).keys())#how many different values j can take
        current_xj_probability = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
        output = output + current_xj_probability
    return output

## Importing Iris data and make the data perfect for classification

In [8]:
def makeLabelled(column):
    second_limit = column.mean()
    first_limit = 0.5 * second_limit
    third_limit = 1.5*second_limit
    for i in range (0,len(column)):
        if (column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] = 2
        else:
            column[i] = 3
    return column


In [9]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [10]:
for i in range(0,X.shape[-1]):
    X[:,i] = makeLabelled(X[:,i])

In [11]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size=0.25,random_state=0)

In [19]:
dictionary = fit(X_train,Y_train)

In [22]:
Y_pred = predict(dictionary,X_test)

In [23]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
